https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE149683

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

from celloracle import motif_analysis as ma
import celloracle as co



import pybedtools

# Add the directory to PATH
os.environ['PATH'] += os.pathsep + os.path.expanduser('~/.conda/envs/my_cell_env/bin')

# Re-import pybedtools to ensure it picks up the new PATH
import pybedtools

# Check if BEDTools is installed and in PATH
if shutil.which("bedtools") is None:
    raise EnvironmentError("BEDTools is not installed or not in the PATH. Please install it and add it to the PATH.")

#conda install -c bioconda bedtools


which: no R in (/usr/local/cuda-11.8/bin:/apps/anaconda/anaconda3/23.5.2/bin:/cm/local/apps/environment-modules/4.5.3//bin:/usr/local/bin:/cm/local/apps/environment-modules/4.5.3/bin:/home/msai/riemerpi001/.vscode-server/cli/servers/Stable-b58957e67ee1e712cebf466b995adf4c5307b2bd/server/bin/remote-cli:/cm/local/apps/environment-modules/4.5.3/bin:/home/msai/riemerpi001/.conda/envs/my_cell_env/bin:/apps/anaconda3/condabin:/cm/local/apps/gcc/11.2.0/bin:/home/msai/riemerpi001/.local/bin:/home/msai/riemerpi001/bin:/cm/local/apps/environment-modules/4.5.3/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/msai/riemerpi001/bin:/usr/bin:/usr/sbin:/cm/shared/apps/slurm/current/sbin:/tc2share/tc-scripts:/usr/share/centrifydc/bin:.:/sbin:/usr/sbin:/cm/local/apps/environment-modules/4.5.3/bin:/opt/dell/srvadmin/bin:/home/msai/riemerpi001/bin:/usr/bin:/usr/sbin:/cm/shared/apps/slurm/current/sbin:/tc2share/tc-scripts:/usr/share/centrifydc/bin:.:/cm/local/apps/environment-modules/4.5.3/bin:/

In [2]:
%config InlineBackend.figure_format = 'retina'

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

# setting peaks 


In [3]:
# Load scATAC-seq peak list.
peaksdir = "/home/msai/riemerpi001/MscProjectDataAnalysis/data/cicero/output"
ref_genome = "hg38"

peaks_df = pd.read_csv(os.path.join(peaksdir, "unique_peaks.csv"), header=0)
# peaks = peaks_df.values.flatten()
peaks_df.head()
# print(peaks[:10
peaks = peaks_df.Peak1.values

peaks = [peak.replace('-', '_') for peak in peaks]

peaks

['chr1_100000984_100001390',
 'chr1_100001537_100001769',
 'chr1_100009735_100010588',
 'chr1_100011376_100011711',
 'chr1_100014423_100015146',
 'chr1_100015590_100016395',
 'chr1_100016458_100016785',
 'chr1_10002433_10004118',
 'chr1_100031226_100031478',
 'chr1_100031511_100032060',
 'chr1_100032133_100032630',
 'chr1_100036034_100036623',
 'chr1_100039444_100039677',
 'chr1_100039822_100040120',
 'chr1_100042658_100043420',
 'chr1_10004544_10004744',
 'chr1_100046001_100046559',
 'chr1_100050652_100051198',
 'chr1_100052448_100052859',
 'chr1_100056150_100056935',
 'chr1_100064548_100065065',
 'chr1_100065129_100065817',
 'chr1_100066334_100066760',
 'chr1_100066773_100067512',
 'chr1_100080893_100081098',
 'chr1_100081149_100082059',
 'chr1_100085039_100085524',
 'chr1_100086774_100087664',
 'chr1_100101249_100101572',
 'chr1_100101601_100101991',
 'chr1_10010168_10011325',
 'chr1_100102117_100102781',
 'chr1_100110010_100110468',
 'chr1_100110771_100112274',
 'chr1_100113087_100

In [9]:
# Load Cicero coaccessibility scores.
cicero_connections = pd.read_csv(os.path.join(peaksdir, "Astrocytes_cerebrum.csv"))
cicero_connections["Peak1"] = cicero_connections["Peak1"].str.replace('-', '_')
cicero_connections["Peak2"] = cicero_connections["Peak2"].str.replace('-', '_')

# Rename the columns if necessary
cicero_connections.columns = ["Peak1", "Peak2", "coaccess"]

# Inspect the DataFrame to confirm the columns
print(cicero_connections.head())
print(cicero_connections.columns)

                      Peak1                     Peak2  coaccess
0  chr1_100231077_100232553  chr1_100232731_100233603  0.109470
1  chr1_100232731_100233603  chr1_100231077_100232553  0.109470
2  chr1_100232731_100233603  chr1_100233665_100233924  0.312481
3  chr1_100233665_100233924  chr1_100232731_100233603  0.312481
4  chr1_100233665_100233924  chr1_100233936_100234574  0.169252
Index(['Peak1', 'Peak2', 'coaccess'], dtype='object')


In [5]:
ma.SUPPORTED_REF_GENOME

# interesting ones are hg38 and hg19

,species,ref_genome,provider
0,Human,hg38,UCSC
1,Human,hg19,UCSC
2,Mouse,mm39,UCSC
3,Mouse,mm10,UCSC
4,Mouse,mm9,UCSC
5,S.cerevisiae,sacCer2,UCSC
6,S.cerevisiae,sacCer3,UCSC
7,Zebrafish,danRer7,UCSC
8,Zebrafish,danRer10,UCSC
9,Zebrafish,danRer11,UCSC


In [6]:
tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome=ref_genome)

tss_annotated.tail()

que bed peaks: 906808
tss peaks in que: 19312


,chr,start,end,gene_short_name,strand
19307,chr2,85184258,85186638,TCF7L1-IT1,+
19308,chr14,71931601,71931891,RGS6,+
19309,chr9,122228480,122228852,LHX6,-
19310,chr21,30486999,30487591,KRTAP19-2,-
19311,chr21,30487957,30488385,KRTAP19-2,-


In [10]:
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated,
                                               cicero_connections=cicero_connections)
print(integrated.shape)
integrated.head()

(21721, 3)


,peak_id,gene_short_name,coaccess
0,chr10_100185264_100186145,ERLIN1,1.0
1,chr10_100186683_100187253,ERLIN1,1.0
2,chr10_100229586_100229991,CHUK,1.0
3,chr10_100237457_100237789,SNORA12,1.0
4,chr10_100329513_100331140,PKD2L1,1.0


In [11]:
peak = integrated[integrated.coaccess >= 0.8]
peak = peak[["peak_id", "gene_short_name"]].reset_index(drop=True)

In [12]:
print(peak.shape)
peak.head()

(18327, 2)


,peak_id,gene_short_name
0,chr10_100185264_100186145,ERLIN1
1,chr10_100186683_100187253,ERLIN1
2,chr10_100229586_100229991,CHUK
3,chr10_100237457_100237789,SNORA12
4,chr10_100329513_100331140,PKD2L1


In [14]:
peak.to_csv(os.path.join(peaksdir,"processed_peak_file.csv"))

https://morris-lab.github.io/CellOracle.documentation/notebooks/01_ATAC-seq_data_processing/option1_scATAC-seq_data_analysis_with_cicero/02_preprocess_peak_data.html

In [23]:
# need to download the ref genome for some reason now...? 

genome_installation = ma.is_genome_installed(ref_genome=ref_genome, genomes_dir=None)
print(ref_genome, "installation: ", genome_installation)

if not genome_installation:
    import genomepy
    genomepy.install_genome(name=ref_genome, provider="UCSC", genomes_dir=None)
else:
    print(ref_genome, "is installed.")

genome hg38 is not installed in this environment.
Please install genome using genomepy.
e.g.
    >>> import genomepy
    >>> genomepy.install_genome(name="hg38", provider="UCSC")
hg38 installation:  False


20:02:58 | INFO | Downloading assembly summaries from UCSC
20:03:06 | INFO | Downloading genome from UCSC. Target URL: https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz...


Download:   0%|          | 0.00/938M [00:00<?, ?B/s]

20:10:15 | INFO | Genome download successful, starting post processing...
20:10:36 | INFO | name: hg38
20:10:36 | INFO | local name: hg38
20:10:36 | INFO | fasta: /home/msai/riemerpi001/.local/share/genomes/hg38/hg38.fa


Filtering Fasta: 0.00 lines [00:00, ? lines/s]

In [24]:
peaks = ma.check_peak_format(peak, ref_genome, genomes_dir=None)

Peaks before filtering:  18327
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  18327


In [25]:
# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks,
                ref_genome=ref_genome,
                genomes_dir=None)

# tfi.scan(motifs=motifs)



In [26]:
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02,
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True)

No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 



2024-05-31 20:16:27,613 - DEBUG - using background: genome hg38 with size 200


Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 



2024-05-31 20:16:32,443 - DEBUG - determining FPR-based threshold


Motif scan started .. It may take long time.



Scanning:   0%|          | 0/17621 [00:00<?, ? sequences/s]

In [27]:
tfi.to_hdf5(file_path="test1.celloracle.tfinfo")

We have the score for each sequence and motif_id pair. In the next step we will filter the motifs with low scores.



In [28]:
tfi.scanned_df.head()

,seqname,motif_id,factors_direct,factors_indirect,score,pos,strand
0,chr10_100185264_100186145,GM.5.0.Mixed.0001,,"EGR1, SRF",8.228200,822,-1
1,chr10_100185264_100186145,GM.5.0.Mixed.0001,,"EGR1, SRF",8.016698,714,-1
2,chr10_100185264_100186145,GM.5.0.Mixed.0001,,"EGR1, SRF",7.981356,766,-1
3,chr10_100185264_100186145,GM.5.0.Mixed.0001,,"EGR1, SRF",7.382178,770,-1
4,chr10_100185264_100186145,GM.5.0.Mixed.0001,,"EGR1, SRF",6.998731,713,-1


In [29]:
# Reset filtering
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10) # not sure what this threshold represents. 

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

df = tfi.to_dataframe()
df.head()


Filtering finished: 5447641 -> 1139977
1. Converting scanned results into one-hot encoded dataframe.


  0%|          | 0/17602 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/12910 [00:00<?, ?it/s]

  0%|          | 0/1094 [00:00<?, ?it/s]

,peak_id,gene_short_name,9430076C15RIK,AC002126.6,AC012531.1,AC226150.2,AFP,AHR,AHRR,AIRE,...,ZNF784,ZNF8,ZNF816,ZNF85,ZSCAN10,ZSCAN16,ZSCAN22,ZSCAN26,ZSCAN31,ZSCAN4
0,chr10_100185264_100186145,ERLIN1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,chr10_100186683_100187253,ERLIN1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr10_100229586_100229991,CHUK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr10_100237457_100237789,SNORA12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,chr10_100329513_100331140,PKD2L1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df.shape # so i'm checking for about 1096 genes? # if over 3K genes might pose problems downstream

assert df.shape[1] < 3000

In [30]:
# Save result as a dataframe
df = tfi.to_dataframe()
df.to_parquet("base_GRN_dataframe.parquet")

https://morris-lab.github.io/CellOracle.documentation/notebooks/02_motif_scan/02_atac_peaks_to_TFinfo_with_celloracle_20200801.html#Notebook-file

In [37]:
oracle = co.Oracle()

In [ ]:
# here I think I need the scRNA-seq data to be loaded... 

# which I think I should do from the .loom or adata file? 

In [38]:
oracle.import_TF_data(TF_info_matrix=df)

ValueError: Please import scRNA-seq data first.